<a href="https://colab.research.google.com/github/Priya1705/CNN_mnist/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
pip install keras-tuner

In [ ]:
from keras_tuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.856333315372467

Best val_accuracy So Far: 0.9096666574478149
Total elapsed time: 00h 03m 10s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
# model.fit(train_images, train_labels, epochs=10, validation_split=0.2, initial_epoch=5)

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0813 - accuracy: 0.9744 - val_loss: 0.3386 - val_accuracy: 0.9141
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0446 - accuracy: 0.9847 - val_loss: 0.4057 - val_accuracy: 0.9130
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0353 - accuracy: 0.9873 - val_loss: 0.4960 - val_accuracy: 0.9092
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0308 - accuracy: 0.9890 - val_loss: 0.5548 - val_accuracy: 0.9099
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0261 - accuracy: 0.9904 - val_loss: 0.6226 - val_accuracy: 0.9059
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0253 - accuracy: 0.9916 - val_loss: 0.5993 - val_accuracy: 0.9094
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0220 - accuracy: 0.9919 - val_loss: 0.6907 - val_accuracy:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

313/313 - 1s - loss: 0.8260 - accuracy: 0.9104
